# Demográficos
Este colab contém o treinamento dos modelos para os demográficos de Rat, Shr e Rch. Por serem entregáveis extras, aplicamos apenas os melhores parâmetros para Rat | Total Domícilios, que foi nosso modelo principal até o momento. Logo, não realizamos random search nem cross-validation por limitação de tempo e pelo caráter não mandatório desta etapa.

In [2]:
# Importando as bibliotecas necessárias para todo o notebook
import pandas as pd

In [3]:
!gdown '1Qcm_0Nki9wiJZ1GCKMR7A53oUC5zPpls'
df = pd.read_csv('emissora_todas_audiencias.csv')

Downloading...
From: https://drive.google.com/uc?id=1Qcm_0Nki9wiJZ1GCKMR7A53oUC5zPpls
To: /content/emissora_todas_audiencias.csv
100% 79.2M/79.2M [00:00<00:00, 198MB/s]


## Preparação de dados

Novamente, precisamos preparar a planilha com todas as colunas de demográficos para Rat, Shr e Rch. Em resumo, realizamos as mesmas manipulações do nosso modelo principal, porém as aplicamos para os outros campos. Para isso, começamos separando os dias e meses em campos individuais:

In [ ]:
def segregate_day(df):
  # Guarda datas da coluna 'Data' como objetos na variável 'dates'
  dates = pd.to_datetime(df['Data'])
  days = []
  # Este loop adiciona o dia de cada data em um array 'days'
  for date in dates:
    days.append(date.day)
  # Cria coluna 'Dia' com array 'days'
  df['Dia'] = days
  # Retorna df modificada
  return df

def segregate_month(df):
  # Guarda datas da coluna 'Data' como objetos na variável 'dates'
  dates = pd.to_datetime(df['Data'])
  months = []
  # Este loop adiciona o mês de cada data em um array 'months'
  for date in dates:
    months.append(date.month)
  # Cria coluna 'mês' com array 'months'
  df['Mês'] = months
  # Salva em CSV na pasta correta
  return df

df = segregate_day(df)
df = segregate_month(df)

Também criamos a coluna binária de feriados:

In [ ]:
from datetime import date # Importa objeto de datas
import holidays # Importa biblioteca de feriados

holidays = holidays.Brazil() # Cria objeto de feriados brasileiros

# Esta função checa o valor de 'Data' de certa linha e, se ele estiver associado
# a um feriado do objeto 'holidays', retorna 1. Caso contrário, retorna 0.
def feriado(row): 
  if pd.to_datetime(row['Data']) in holidays:
    return 1
  return 0

# A função apply aplica a função "feriado" a todas as linhas da dataframe.
# O valor retornado é adicionado a uma nova coluna chamada "Feriado".
# Desse modo, datas de feriado ficam com 1 nessa coluna, enquanto as outras ficam
# com 0
df['Feriado'] = df.apply(lambda row: feriado(row), axis=1)

Depois, fazemos o one-hot encoding das categorias:

In [ ]:
# Este método transforma os valores únicos de uma coluna em colunas individuais auxiliares,
# seguindo o modelo de one-hot encoding. O parâmetro "prefix" determina o prefixo de cada coluna.
dummies = pd.get_dummies(df['Categoria'], prefix='Categoria')

# Concatena colunas auxiliares com planilha existente
df = pd.concat([df, dummies], axis=1)

# Deleta coluna de categorias original
df.pop('Categoria')

0         JORNALISMO
1         JORNALISMO
2         JORNALISMO
3         JORNALISMO
4         JORNALISMO
             ...    
218846    JORNALISMO
218847    JORNALISMO
218848    JORNALISMO
218849    JORNALISMO
218850    JORNALISMO
Name: Categoria, Length: 218851, dtype: object

Então, destrinchamos a coluna de 'Hora Início' em 'Hora' e 'Minuto'.

In [ ]:
series = df['Hora Início'] # Isola coluna de horários
hours = [] # Inicia array para guardar unidades de hora
minutes = [] # Inicia array para guardar unidades de minuto

# Este loop passa por cada item da coluna de horário, acessa as unidades através
# dos indexes do valor em string, transforma em inteiro e salva no array
for time in series:
  hours.append(int(time[:2])) # A hora corresponde aos caracteres 0 e 1
  # O caractere 2 é ":" no formato "xx:xx:xx"
  minutes.append(int(time[3:5])) # O minuto corresponde aos caracteres 3 e 4

# Os arrays são adicionados como colunas na dataframe
df['Hora'] = hours
df['Minuto'] = minutes

Por fim, fizemos o label encoding dos dias da semana e deletamos as colunas desnecessárias.

In [ ]:
# Hashmap de associação de dias da semana com números
week_encoding = {'Segunda': 1, 'Terça': 2, 'Quarta': 3, 'Quinta': 4, 'Sexta': 5, 
                 'Sábado': 6, 'Domingo': 7}

# Faz a substituição com base no hashmap
df['Dia da Semana'] = df['Dia da Semana'].replace(week_encoding)


In [ ]:
df.drop(['Data', 'Hora Início', 'Emissora', 'Programa'], axis=1)

,Dia da Semana,Total Domicílios | Rat%,AB | Rat%,C1 | Rat%,C2 | Rat%,DE | Rat%,Masculino | Rat%,Feminino | Rat%,4-11 anos | Rat%,12-17 anos | Rat%,...,Categoria_POLITICO,Categoria_PREMIACAO,Categoria_REALITY SHOW,Categoria_RELIGIOSO,Categoria_REPORTAGEM,Categoria_RURAL,Categoria_SERIES,Categoria_SHOW,Hora,Minuto
0,1,4.27,2.40,0.31,0.82,5.23,1.65,2.35,0.0,0.00,...,0,0,0,0,0,0,0,0,6,0
1,1,4.89,3.02,0.51,1.08,5.23,1.89,2.75,0.0,0.00,...,0,0,0,0,0,0,0,0,6,5
2,1,5.74,3.13,1.53,1.22,5.23,1.97,3.24,0.0,1.22,...,0,0,0,0,0,0,0,0,6,10
3,1,6.55,3.06,1.86,1.48,6.46,1.92,3.92,0.0,1.52,...,0,0,0,0,0,0,0,0,6,15
4,1,7.27,3.49,2.18,1.48,6.46,2.23,4.05,0.0,1.52,...,0,0,0,0,0,0,0,0,6,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218846,4,1.67,0.30,0.41,1.03,0.53,0.17,0.92,0.0,0.96,...,0,0,0,0,0,0,0,0,29,35
218847,4,1.97,0.30,0.41,1.45,0.35,0.17,1.07,0.0,0.96,...,0,0,0,0,0,0,0,0,29,40
218848,4,2.22,0.30,0.41,1.45,0.88,0.17,1.26,0.0,0.96,...,0,0,0,0,0,0,0,0,29,45
218849,4,2.71,0.54,0.41,1.81,0.88,0.37,1.42,0.0,0.96,...,0,0,0,0,0,0,0,0,29,50


## Rat%

Com isso, podemos dividir os dados em treino e teste. Nesse caso, temos como 'y' todas as colunas de Rat%, exceto a Total Domicílios, dado que essa já é contemplada em nosos modelo principal.

In [ ]:
from sklearn.model_selection import train_test_split

model = df.copy() # Copia dataframe

# Dividindo x e y
y = model[['AB | Rat%', 'C1 | Rat%', 'C2 | Rat%',
       'DE | Rat%', 'Masculino | Rat%', 'Feminino | Rat%', '4-11 anos | Rat%',
       '12-17 anos | Rat%', '18-24 anos | Rat%', '25-34 anos | Rat%',
       '35-49 anos | Rat%', '50-59 anos | Rat%', '60+ anos | Rat%']].values # Score de audiência
x = model[['Dia da Semana',
       'Mês', 'Dia', 'Hora', 'Minuto', 'Feriado',  'Categoria_AUDITORIO', 'Categoria_CARROS E MOTORES',
       'Categoria_CULINARIO', 'Categoria_DEBATE', 'Categoria_DOCUMENTARIO',
       'Categoria_EDUCATIVO', 'Categoria_ENTREVISTA', 'Categoria_ESPORTE',
       'Categoria_FEMININO', 'Categoria_FILME', 'Categoria_FUTEBOL',
       'Categoria_GAME SHOW', 'Categoria_HUMORISTICO', 'Categoria_JORNALISMO',
       'Categoria_MINISSERIE', 'Categoria_MUSICAL', 'Categoria_NOVELA',
       'Categoria_POLITICO', 'Categoria_PREMIACAO', 'Categoria_REALITY SHOW',
       'Categoria_RELIGIOSO', 'Categoria_REPORTAGEM', 'Categoria_RURAL',
       'Categoria_SERIES', 'Categoria_SHOW']].values # Features de data, hora e categoria

# Dividindo dados para treino e dados para teste
x_train, x_test, y_train, y_test = train_test_split(x, y, 
                                                    test_size = 0.3, 
                                                    random_state = 42)

Com isso, criamos o modelo de árvore decisória com profundidade entre 20 e 30, conforme selecionado em nosso modelo principal.

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

dt = DecisionTreeRegressor(max_depth=24) 
dt.fit(x_train, y_train.squeeze()) # Cria decision tree com base em dados de treino
y_pred = dt.predict(x_test) # Faz predições para dataset de test

# Erro médio absoluto
mean_absolute = mean_absolute_error(y_test, y_pred)
print("Erro médio: " + str(mean_absolute))

# Erro médio absoluto em porcentagem (erro / média), arredondado para duas casas
# decimais para facilitar a visualização
percentual_error = str(round((mean_absolute/y_test.mean() * 100), 2))
print("Erro médio percentual: " + percentual_error + "%")

# R²
r2 = r2_score(y_test, y_pred)
print("R²: " + str(r2))

# R² ajustado
r2_adjusted = 1 - (1 - r2) * (len(y) - 1) / (len(y) - x.shape[1] - 1)
print("R² ajustado: " + str(r2))

Erro médio: 0.6161550035799196
Erro médio percentual: 12.54%
R²: 0.9046298602404631
R² ajustado: 0.9046298602404631


Por fim, salvamos o modelo em txt utilizando a biblioteca 'pickle'. Assim, poderemos utilizá-lo para fazer predições futuramente sem retreiná-lo.

In [ ]:
import pickle
pickle.dump(dt, open('/content/drive/MyDrive/Sprint 4/modelos/all_rat.txt', 'wb'))

## Shr%

Seguimos o mesmo processo para a métrica de Shr%.

In [ ]:
from sklearn.model_selection import train_test_split

model = df.copy() # Copia dataframe

# Dividindo x e y
y = model[['Total Domicílios | Shr%', 'AB | Shr%', 'C1 | Shr%', 'C2 | Shr%',
       'DE | Shr%', 'Masculino | Shr%', 'Feminino | Shr%', '4-11 anos | Shr%',
       '12-17 anos | Shr%', '18-24 anos | Shr%', '25-34 anos | Shr%',
       '35-49 anos | Shr%', '50-59 anos | Shr%', '60+ anos | Shr%']].values # Score de audiência
x = model[['Dia da Semana',
       'Mês', 'Dia', 'Hora', 'Minuto', 'Feriado',  'Categoria_AUDITORIO', 'Categoria_CARROS E MOTORES',
       'Categoria_CULINARIO', 'Categoria_DEBATE', 'Categoria_DOCUMENTARIO',
       'Categoria_EDUCATIVO', 'Categoria_ENTREVISTA', 'Categoria_ESPORTE',
       'Categoria_FEMININO', 'Categoria_FILME', 'Categoria_FUTEBOL',
       'Categoria_GAME SHOW', 'Categoria_HUMORISTICO', 'Categoria_JORNALISMO',
       'Categoria_MINISSERIE', 'Categoria_MUSICAL', 'Categoria_NOVELA',
       'Categoria_POLITICO', 'Categoria_PREMIACAO', 'Categoria_REALITY SHOW',
       'Categoria_RELIGIOSO', 'Categoria_REPORTAGEM', 'Categoria_RURAL',
       'Categoria_SERIES', 'Categoria_SHOW']].values # Features de data, hora e categoria

# Dividindo dados para treino e dados para teste
x_train, x_test, y_train, y_test = train_test_split(x, y, 
                                                    test_size = 0.3, 
                                                    random_state = 42)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

dt = DecisionTreeRegressor(max_depth=28) 
dt.fit(x_train, y_train.squeeze()) # Cria decision tree com base em dados de treino
y_pred = dt.predict(x_test) # Faz predições para dataset de test

# Erro médio absoluto
mean_absolute = mean_absolute_error(y_test, y_pred)
print("Erro médio: " + str(mean_absolute))

# Erro médio absoluto em porcentagem (erro / média), arredondado para duas casas
# decimais para facilitar a visualização
percentual_error = str(round((mean_absolute/y_test.mean() * 100), 2))
print("Erro médio percentual: " + percentual_error + "%")

# R²
r2 = r2_score(y_test, y_pred)
print("R²: " + str(r2))

# R² ajustado
r2_adjusted = 1 - (1 - r2) * (len(y) - 1) / (len(y) - x.shape[1] - 1)
print("R² ajustado: " + str(r2))

Erro médio: 3.3815761797925536
Erro médio percentual: 11.42%
R²: 0.8640319352292989
R² ajustado: 0.8640319352292989


In [ ]:
import pickle
pickle.dump(dt, open('/content/drive/MyDrive/Sprint 4/modelos/share.txt', 'wb'))

## Rch%

O processo é repetido para as colunas de Rch.

In [ ]:
# Esta função acessa cada valor da coluna de audiência, transforma-o em string,
# substitui vírgulas por pontos e volta o tipo para float
df['Total Indivíduos | Rch%'] = df['Total Indivíduos | Rch%'].apply(lambda x: float(str(x).replace(',', '.')))

In [ ]:
from sklearn.model_selection import train_test_split

model = df.copy() # Copia dataframe

# Dividindo x e y
y = model[['Total Indivíduos | Rch%', 'AB | Rch%', 'C1 | Rch%', 'C2 | Rch%',
       'DE | Rch%', 'Masculino | Rch%', 'Feminino | Rch%', '4-11 anos | Rch%',
       '12-17 anos | Rch%', '18-24 anos | Rch%', '25-34 anos | Rch%',
       '35-49 anos | Rch%', '50-59 anos | Rch%', '60+ anos | Rch%']].values # Score de audiência
x = model[['Dia da Semana',
       'Mês', 'Dia', 'Hora', 'Minuto', 'Feriado',  'Categoria_AUDITORIO', 'Categoria_CARROS E MOTORES',
       'Categoria_CULINARIO', 'Categoria_DEBATE', 'Categoria_DOCUMENTARIO',
       'Categoria_EDUCATIVO', 'Categoria_ENTREVISTA', 'Categoria_ESPORTE',
       'Categoria_FEMININO', 'Categoria_FILME', 'Categoria_FUTEBOL',
       'Categoria_GAME SHOW', 'Categoria_HUMORISTICO', 'Categoria_JORNALISMO',
       'Categoria_MINISSERIE', 'Categoria_MUSICAL', 'Categoria_NOVELA',
       'Categoria_POLITICO', 'Categoria_PREMIACAO', 'Categoria_REALITY SHOW',
       'Categoria_RELIGIOSO', 'Categoria_REPORTAGEM', 'Categoria_RURAL',
       'Categoria_SERIES', 'Categoria_SHOW']].values # Features de data, hora e categoria

# Dividindo dados para treino e dados para teste
x_train, x_test, y_train, y_test = train_test_split(x, y, 
                                                    test_size = 0.3, 
                                                    random_state = 42)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

dt = DecisionTreeRegressor(max_depth=28) 
dt.fit(x_train, y_train.squeeze()) # Cria decision tree com base em dados de treino
y_pred = dt.predict(x_test) # Faz predições para dataset de test

# Erro médio absoluto
mean_absolute = mean_absolute_error(y_test, y_pred)
print("Erro médio: " + str(mean_absolute))

# Erro médio absoluto em porcentagem (erro / média), arredondado para duas casas
# decimais para facilitar a visualização
percentual_error = str(round((mean_absolute/y_test.mean() * 100), 2))
print("Erro médio percentual: " + percentual_error + "%")

# R²
r2 = r2_score(y_test, y_pred)
print("R²: " + str(r2))

# R² ajustado
r2_adjusted = 1 - (1 - r2) * (len(y) - 1) / (len(y) - x.shape[1] - 1)
print("R² ajustado: " + str(r2))

Erro médio: 0.602154251925767
Erro médio percentual: 12.56%
R²: 0.8965005312237694
R² ajustado: 0.8965005312237694


In [ ]:
import pickle
pickle.dump(dt, open('/content/drive/MyDrive/Sprint 4/modelos/rch.txt', 'wb'))